In [1]:
# summaflow library
from summaflow import (
    Stats,
    GeoLayer,
    SUMMAWorkflow,
)

# 3rd party libraries
import pandas as pd
import numpy as np
import geopandas as gpd
import xarray as xr

# built-in libraries
import os

In [2]:
# paths
root_path_layers = '/Users/kasrakeshavarz/Documents/github-repos/summa-model-specific/tests/notebooks/test-stats/'

# layers' paths
landcover_path = os.path.join(root_path_layers, 'landsat-landcover')
soilclass_path = os.path.join(root_path_layers, 'usda-soil')
merithdyr_path = os.path.join(root_path_layers, 'merit-hydro')

# geolayer's path
root_path_geoms = '../../../examples/bow-at-calgary-geofabric/'
riv_path = os.path.join(root_path_geoms, 'bcalgary_rivers.shp')
cat_path = os.path.join(root_path_geoms, 'bcalgary_subbasins.shp')
hru_path = os.path.join(root_path_geoms, 'bcalgary_subbasins.shp')

____

## Building GeoLayers

In [3]:
# layers needed by the setup workflow
# elevation
elv = GeoLayer.from_maf(
    maf_stats=os.path.join(merithdyr_path, 'summaflow_stats_elv.csv'),
    maf_layer=os.path.join(merithdyr_path, 'summaflow_elv.tif'),
    maf_geolayer=os.path.join(cat_path),
    unit = 'meters',
)
# landcover
landcover = GeoLayer.from_maf(
    maf_stats=os.path.join(landcover_path, 'summaflow_stats_NA_NALCMS_landcover_2020_30m.csv'),
    maf_layer=os.path.join(landcover_path, 'summaflow_NA_NALCMS_landcover_2020_30m.tif'),
    maf_geolayer=os.path.join(cat_path),
    unit = 'dimensionless',
)
# USDA soil classes
soil = GeoLayer.from_maf(
    maf_stats=os.path.join(soilclass_path, 'summaflow_stats_soil_classes.csv'),
    maf_layer=os.path.join(soilclass_path, 'summaflow_soil_classes.tif'),
    maf_geolayer=os.path.join(cat_path),
    unit = 'dimensionless',
)

In [4]:
elv

Stats: {'mean'}
Layer: True
Geometry: True
Geolayer Unit: meter

In [5]:
landcover

Stats: {'mean', 'min', 'stdev', 'count', 'variance', 'q', 'variety', 'coefficient_of_variation', 'median', 'minority', 'max', 'majority', 'frac'}
Layer: True
Geometry: True
Geolayer Unit: dimensionless

In [6]:
soil

Stats: {'mean', 'min', 'median', 'minority', 'max', 'majority', 'frac'}
Layer: True
Geometry: True
Geolayer Unit: dimensionless

____

## Reading Geometries

In [7]:
riv_obj = gpd.read_file(riv_path)
cat_obj = gpd.read_file(cat_path)
hru_obj = gpd.read_file(hru_path)

____

In [8]:
forcing_root = '/Users/kasrakeshavarz/Documents/github-repos/summa-model-specific/tests/notebooks/forcings'

exp = SUMMAWorkflow(
    forcing_data = [
        os.path.join(forcing_root, 'Bow_at_Banff_ERA5_remapped_domain_Bow_at_Banff_ERA5_merged_201101.nc')
        ],
    forcing_name_mapping = {
        'pptrate': 'precipitation',
        'airtemp': 'airtemp',
    },
    forcing_unit_mapping = {
        'precipitation': 'meters/hour',
        'airtemp': 'degC',
    },
    forcing_to_unit_mapping = {
        'precipitation': 'mm/s',
        'airtemp': 'K',
    },
    forcing_attrs = {
        'measurement_height': 40,
        'measurement_height_unit': 'meter',
        'forcing_time_zone': 'utc', # original timezone of the forcing datatset
        'target_time_zone': 'europe/vienna', # set 'utc' for large domain crossing multiple time-zones
        'local': {
            'pr': {
                'long_name': 'precipitation at the surface',
            },
            'temp': {
                'long_name': 'Air temperature',
            },
        },
        'global': {},
    },
    topology_data = {
        'riv': riv_obj,
        'hru': hru_obj,
        'cat': cat_obj,
    },
    topology_unit_mapping = {'a': 'm', 'c': 'ha'},
    topology_to_unit_mapping = {'a': 'bar', 'c': 'dca'},
    topology_attrs = {
        'gru_fid': 'COMID',
        'hru_fid': 'COMID',
    },
    geospatial_data = {
        'elevation': elv,
        'soilTypeIndex': soil,
        'vegTypeIndex': landcover,
    },
    settings = {
        'parallel': False,
    },
)

____

## Manual tests on forcing files

In [9]:
ds = xr.open_dataset('../forcings/Bow_at_Banff_ERA5_remapped_domain_Bow_at_Banff_ERA5_merged_201101.nc')

In [10]:
ds.assign_coords({
    'time': ds.time.to_index().tz_localize('utc').tz_convert('europe/vienna').tz_localize(None)
}).time

<xarray.DataArray 'time' (time: 744)> Size: 6kB
array(['2011-01-01T01:00:00.000000000', '2011-01-01T02:00:00.000000000',
       '2011-01-01T03:00:00.000000000', ..., '2011-01-31T22:00:00.000000000',
       '2011-01-31T23:00:00.000000000', '2011-02-01T00:00:00.000000000'],
      shape=(744,), dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 6kB 2011-01-01T01:00:00 ... 2011-02-01

____

## Workflow tests

In [11]:
exp.init_forcing()

In [12]:
exp._forcing_attrs

{'measurement_height': 40,
 'measurement_height_unit': 'meter',
 'forcing_time_zone': 'utc',
 'target_time_zone': 'europe/vienna',
 'local': {'pr': {'long_name': 'precipitation at the surface'},
  'temp': {'long_name': 'Air temperature'}},
 'global': {},
 'tz_info': 'localTime'}